In [118]:
import geopandas as gpd
import pandas as pd
import geemap
import ee


In [504]:
cropfile = 'pabbi_crop.geojson'
gdf = gpd.read_file(cropfile)
#gdf.head()

In [812]:
# Filtering the composite labeling
#step 1: Remove rows with '/' in 'Crop_Type'
gdf_filtered = gdf[~gdf['Crop_Type'].fillna('').str.contains('/')]

#Step 2: Remove the nocrop type from the filtered data
nocrop = ['No crop' ,'Mix', 'Trees', 'Buitup' ,'Water body', 'Water Channel','Other','Builtup','Barren','Barley','Garlic','Egg Plant','Potato','Indian Squash','Sugarcane' ]
gdf_filtered = gdf_filtered[~gdf_filtered['Crop_Type'].isin(nocrop)]

#Step 3: Final filtered data
gdf_filtered['Crop_Type'].value_counts()
print(gdf_filtered['Crop_Type'].value_counts())
print ('Mean of Crop_Type',gdf_filtered['Crop_Type'].value_counts().mean())

# 0) Drop any rows with missing Crop_Type
gdf_filtered = gdf_filtered.dropna(subset=['Crop_Type'])

Crop_Type
Tomato            94
Wheat             81
Persian Clover    57
Corn              16
Orchard            6
Eucalyptus         5
Lady Finger        5
Name: count, dtype: int64
Mean of Crop_Type 37.714285714285715


In [791]:
# 1) Identify major vs. minor
major_crop = ['Wheat','Persian Clover','Tomato']
all_crops  = gdf_filtered['Crop_Type'].unique().tolist()
minor_crop = [c for c in all_crops if c not in major_crop]
print('Major Crop:', major_crop)
print('Minor Crop:', minor_crop)

Major Crop: ['Wheat', 'Persian Clover', 'Tomato']
Minor Crop: ['Corn', 'Orchard', 'Eucalyptus', 'Lady Finger']


In [792]:
# 2) Compute overall mean of majors, set targets
counts    = gdf_filtered['Crop_Type'].value_counts()
mean_major = counts[major_crop].mean()            # avg size of the two majors
target_min = int(mean_major * 0.6)                # 60% of that for minors
print(f"Major average = {mean_major:.1f}, target for minors = {target_min}")


Major average = 77.3, target for minors = 46


In [813]:
from sklearn.utils import resample


# 3) Build per-class target_counts
target_counts = {}
for crop, cnt in gdf_filtered['Crop_Type'].value_counts().items():
    if crop == 'Persian Clover':
        target_counts[crop] = int(mean_major * 1.3)
    elif crop == 'Wheat':
        target_counts[crop] = int(mean_major * 1.2)
    elif crop == 'Tomato':
        target_counts[crop] = int(mean_major * 1.4)
    else:
        target_counts[crop] = int(mean_major)    # baseline for everyone else

print("Target sample sizes per class:", target_counts)

Target sample sizes per class: {'Tomato': 108, 'Wheat': 92, 'Persian Clover': 100, 'Corn': 77, 'Orchard': 77, 'Eucalyptus': 77, 'Lady Finger': 77}


In [814]:
# 4) Remove any crops that are already larger than target
# 4) Resample each group to its target
balanced_parts = []
for crop, group in gdf_filtered.groupby('Crop_Type'):
    target = target_counts[crop]
    if len(group) < target:
        # oversample with replacement
        samp = resample(
            group,
            replace=True,
            n_samples=target,
            random_state=42
        )
    else:
        # downsample or keep exact
        samp = group.sample(n=target, random_state=42)
    balanced_parts.append(samp)

gdf_strat = pd.concat(balanced_parts, ignore_index=True)

# 5) Verify
print("\nNew balanced counts:")
print(gdf_strat['Crop_Type'].value_counts())


New balanced counts:
Crop_Type
Tomato            108
Persian Clover    100
Wheat              92
Lady Finger        77
Eucalyptus         77
Corn               77
Orchard            77
Name: count, dtype: int64


In [212]:
ee.Authenticate()

True

In [213]:
ee.Initialize()

In [815]:
# Converting the gdf_balanced to ee feature collection
crop_fc = geemap.gdf_to_ee(gdf_strat)

In [816]:
# Define a function to get imagery an apply filter
def get_imagery (start_date,end_date):
    collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    filtered_collection = collection.filterDate(start_date, end_date).filterBounds(crop_fc).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
    median_image = collection.median().clip(crop_fc)
    # Select the first image from the filtered collection
    median_image = filtered_collection.median().clip(crop_fc)
    return median_image


In [817]:
# Seperate for Rabi and Kharif Season of the current year


def get_ndvindwi(median_image):
    # Calculate NDVI
    ndvi = median_image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    # McFeeters NDWI = (Green - NIR) / (Green + NIR)
    ndwi = median_image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    return ndvi, ndwi

def get_mndwi(median_image):
    # Calculate MNDWI
    mndwi = median_image.normalizedDifference(['B3', 'B11']).rename('MNDWI')
    return mndwi

def get_savi(median_image):
    # Calculate Soil Adjusted Vetetation Index (SAVI)
    savi = median_image.expression(
        '(NIR - RED) / (NIR + RED + L) * (1 + L)', {
            'NIR': median_image.select('B8'),
            'RED': median_image.select('B4'),
            'L': 0.5
        }).rename('SAVI')
    return savi
def get_arvi(median_image):
    # Calculate Atmospherically Resistant Vegetation Index (ARVI)
    arvi = median_image.expression(
        '(NIR - (2 * RED - BLUE)) / (NIR + (2 * RED - BLUE))', {
            'NIR': median_image.select('B8'),
            'RED': median_image.select('B4'),
            'BLUE': median_image.select('B2')
        }).rename('ARVI')
    return arvi

def get_otherbands(median_image):
    # Adding other bans and calculating their mean
    bands = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']
    band_img = median_image.select(bands).rename([f'{b}_mean' for b in bands])
    return band_img

def get_texture(median_image):
    # Calculating the texture
    nir_int = median_image.select('B8').toInt32()
    texture = nir_int.glcmTexture(size=3)
    contrast = texture.select('B8_contrast').rename('contrast')
    entropy = texture.select('B8_ent').rename('entropy')
    return contrast,entropy

def get_features(ndvi, ndwi, band_img, contrast, mndwi,entropy,savi,arvi):
    # Combine all features into a single image
    features_img = band_img.addBands([ndvi, ndwi,mndwi,entropy,savi,arvi])
    features_img = features_img.addBands(contrast)
    return features_img


In [818]:
#---------------------------
# Plan reduciton

def get_reduction(fc):
    # Step 2: Calculate mean NDVI for each agri polygon
    pcs = fc.reduceRegions(
    collection=crop_fc,
    reducer=ee.Reducer.mean(),
    scale=10,)
    # Convert the result to dataframe
    cf= geemap.ee_to_gdf(pcs)
    return cf



In [819]:
#----------------------------------
# Now calculate area, perimeter etc
def get_area(cropgdf):
# We need to calculate the area of each polygon in square meters
# Therefore we will convert the geometry to a projected coordinate system (EPSG:32643)
    cropgdf=cropgdf.to_crs(epsg=32643)
# Calculating the area , perimetere and compactness
    cropgdf['Area_m2'] = cropgdf.geometry.area
    cropgdf['Perimeter_m'] = cropgdf.geometry.length
    cropgdf['Compactness'] = (4 * 3.14 * cropgdf['Area_m2']) / (cropgdf['Perimeter_m'] ** 2)

# Switch back to lat/lon if needed for mapping
    cropgdf = cropgdf.to_crs(epsg=4326)

#Inspect the new columns
#gdfcrop_features[['Area_m2','Perimeter_m','Compactness']].head()
    return cropgdf



In [820]:
# Getting imagery and features
early_Kharif=get_imagery('2025-04-01','2025-05-05')
rabi = get_imagery('2024-10-01','2025-03-31')
full_year = get_imagery('2024-09-01','2025-05-05')

ndvi_kharif, ndwi_kharif = get_ndvindwi(early_Kharif)
ndvi_rabi, ndwi_rabi = get_ndvindwi(rabi)
ndvi_full, ndwi_full = get_ndvindwi(full_year)

savi_kharif = get_savi(early_Kharif)
savi_rabi = get_savi(rabi)
savi_full = get_savi(full_year)

arvi_kharif = get_arvi(early_Kharif)
arvi_rabi = get_arvi(rabi)
arvi_full = get_arvi(full_year)

mndwi_kharif = get_mndwi(early_Kharif)
mndwi_rabi = get_mndwi(rabi)    
mndwi_full = get_mndwi(full_year)

band_kharif = get_otherbands(early_Kharif)
band_rabi = get_otherbands(rabi)
band_full = get_otherbands(full_year)

contrast_kharif,entropy_kharif = get_texture(early_Kharif)
contrast_rabi, entropy_rabi = get_texture(rabi)    
contrast_full,entropy_full = get_texture(full_year)

kharif_feats = get_features(ndvi_kharif, ndwi_kharif,band_kharif, contrast_kharif,entropy_kharif,mndwi_kharif,savi_kharif,arvi_kharif)
rabi_feats =   get_features(ndvi_rabi, ndwi_rabi,band_rabi, contrast_rabi,entropy_rabi,mndwi_rabi,savi_rabi,arvi_rabi)    
full_feats =   get_features(ndvi_full, ndwi_full,band_full, contrast_full,entropy_full,mndwi_full,savi_full,arvi_full)    

composite_features = kharif_feats.addBands(full_feats)

In [821]:
# Get reduction gdf for  Kharif_feas,rabi_feats and composite_features
kharif_reduced=get_reduction(kharif_feats)
rabi_reduced=get_reduction(rabi_feats)
full_reduced=get_reduction(full_feats)

composite_reduced=get_reduction(composite_features)


In [822]:
#Get area, perimetetr and compactness for kharif_reduced,rabi_reduced and composite_reduced
kharif_reduced=get_area(kharif_reduced)
rabi_reduced=get_area(rabi_reduced)
composite_reduced=get_area(composite_reduced)
full_reduced=get_area(full_reduced)

In [823]:
composite_reduced['delta_NDVI'] = composite_reduced['NDVI'] - composite_reduced['NDVI_1']

In [824]:
composite_reduced.columns

Index(['geometry', 'ARVI', 'ARVI_1', 'Area_Acre', 'B11_mean', 'B11_mean_1',
       'B12_mean', 'B12_mean_1', 'B2_mean', 'B2_mean_1', 'B3_mean',
       'B3_mean_1', 'B4_mean', 'B4_mean_1', 'B8_mean', 'B8_mean_1',
       'Crop_Type', 'FFID', 'Landuse_Ma', 'MNDWI', 'MNDWI_1', 'Mouza_Name',
       'NDVI', 'NDVI_1', 'NDWI', 'NDWI_1', 'Parcel_ID', 'SAVI', 'SAVI_1',
       'contrast', 'contrast_1', 'entropy', 'entropy_1', 'Area_m2',
       'Perimeter_m', 'Compactness', 'delta_NDVI'],
      dtype='object')

In [825]:
# Checking one by one

#gdfcrop_features= kharif_reduced # Checking the Kharif Season
#gdfcrop_features= rabi_reduced # Checking the rabi Season
#gdfcrop_features= full_reduced # Checking the full Season
gdfcrop_features= composite_reduced # Checking the composite Season




In [826]:
# List out the exact feature columns in your composite_reduced
feature_cols = [
    # early‑Kharif means (no “_1” suffix)
    'B2_mean', 'B3_mean', 'B4_mean', 'B8_mean', 'B11_mean', 'B12_mean','ARVI',
    'SAVI','NDVI', 'NDWI', 'contrast','MNDWI','entropy',
    # full‑year means (with “_1” suffix)
    'B2_mean_1', 'B3_mean_1', 'B4_mean_1', 'B8_mean_1', 'B11_mean_1', 'B12_mean_1','ARVI_1','SAVI_1',
    'NDVI_1', 'NDWI_1', 'contrast_1', 'MNDWI_1','entropy_1',
    # delta
    'delta_NDVI',
    # shape metrics (if present—you can add Area_Acre or Area_m2 etc.)
    'Area_Acre','Area_m2','Perimeter_m','Compactness'       # or 'Area_m2','Perimeter_m','Compactness'
  
]

#drop_feats = ['B2_mean_1']
#feature_cols = [f for f in feature_cols if f not in drop_feats]


In [827]:
from sklearn.model_selection import train_test_split
# Splitting the data into training and testing sets

#Step 1 Define features (NDVI mean) and target label (Crop_Type)
X = composite_reduced[feature_cols].values
y = composite_reduced['Crop_Type'].values

# Train-test Split 80 % train and 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)



In [833]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Initialize the model
rf_model = RandomForestClassifier(n_estimators=300,
    max_depth=20,
    min_samples_split=2,
    min_samples_leaf=1,
    class_weight='balanced',
    random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_model.predict(X_test)

# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred,zero_division=0))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Classification Report:
                precision    recall  f1-score   support

          Corn       1.00      1.00      1.00        15
    Eucalyptus       1.00      1.00      1.00        15
   Lady Finger       1.00      1.00      1.00        16
       Orchard       1.00      1.00      1.00        15
Persian Clover       0.82      0.90      0.86        20
        Tomato       0.95      0.86      0.90        22
         Wheat       0.84      0.84      0.84        19

      accuracy                           0.93       122
     macro avg       0.94      0.94      0.94       122
  weighted avg       0.94      0.93      0.93       122

Confusion Matrix:
[[15  0  0  0  0  0  0]
 [ 0 15  0  0  0  0  0]
 [ 0  0 16  0  0  0  0]
 [ 0  0  0 15  0  0  0]
 [ 0  0  0  0 18  1  1]
 [ 0  0  0  0  1 19  2]
 [ 0  0  0  0  3  0 16]]


In [ ]:
# Add predictions to gdf_strat (or full set)
gdf_strat['pred'] = rf_model.predict(gdf_strat[feature_cols].values)

# Convert back to EE for mapping
#fc_pred = geemap.gdf_to_ee(gdf_strat, id_property='Parcel_ID')
#Map.addLayer(fc_pred.style(color='pred', palette='viridis'), {}, 'Predicted Crop Types')


In [ ]:
# Save the model prediction to a geojson file
gdf_strat.to_file("pabbi_crop_predictions.geojson", driver="GeoJSON")


In [ ]:
#Export your model and features for reproducibility
#You can pickle your RF and the feature‐extraction parameters so that any future user can apply the exact same classification:

import joblib
joblib.dump({
    'model': rf_model,
    'feature_cols': feature_cols
}, 'crop_rf_pipeline.pkl')


In [829]:
# Hyper Parameter Tuning using RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# 1) Define the parameter search space
param_dist = {
    'n_estimators':       [100, 200, 300, 400, 500],
    'max_depth':          [None, 10, 15, 20, 25],
    'min_samples_split':  [2, 5, 10],
    'min_samples_leaf':   [1, 2, 4, 6],
    'class_weight':       ['balanced', 'balanced_subsample']
}

# 2) Set up the RandomizedSearchCV
rs = RandomizedSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_distributions=param_dist,
    n_iter=30,              # number of combinations to try
    scoring='f1_weighted',  # optimize for weighted F1
    cv=5,                   # 5‑fold cross‑validation
    n_jobs=-1,              # use all cores
    random_state=42,
    verbose=1
)

# 3) Run the search on your training data
rs.fit(X_train, y_train)

# 4) Inspect the best parameters
print("Best hyperparameters found:")
for k, v in rs.best_params_.items():
    print(f"  {k}: {v}")

# 5) Evaluate the best estimator on the test set
best_rf = rs.best_estimator_
y_pred = best_rf.predict(X_test)
print("\nClassification Report (best estimator):")
print(classification_report(y_test, y_pred, zero_division=0))


Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best hyperparameters found:
  n_estimators: 300
  min_samples_split: 2
  min_samples_leaf: 1
  max_depth: None
  class_weight: balanced

Classification Report (best estimator):
                precision    recall  f1-score   support

          Corn       1.00      1.00      1.00        15
    Eucalyptus       1.00      1.00      1.00        15
   Lady Finger       1.00      1.00      1.00        16
       Orchard       1.00      1.00      1.00        15
Persian Clover       0.82      0.90      0.86        20
        Tomato       0.95      0.86      0.90        22
         Wheat       0.84      0.84      0.84        19

      accuracy                           0.93       122
     macro avg       0.94      0.94      0.94       122
  weighted avg       0.94      0.93      0.93       122



In [595]:

importances = rf_model.feature_importances_
for name, imp in sorted(zip(feature_cols, importances), key=lambda x: -x[1]):
    print(f"{name}: {imp:.3f}")


B3_mean: 0.060
contrast_1: 0.054
B3_mean_1: 0.053
B11_mean: 0.048
B8_mean: 0.047
MNDWI: 0.045
B12_mean_1: 0.041
contrast: 0.040
B11_mean_1: 0.036
Perimeter_m: 0.035
MNDWI_1: 0.034
Area_Acre: 0.032
B12_mean: 0.032
B8_mean_1: 0.031
NDWI_1: 0.031
Compactness: 0.031
Area_m2: 0.030
B2_mean: 0.029
ARVI: 0.027
B4_mean: 0.027
B4_mean_1: 0.025
NDWI: 0.025
NDVI: 0.024
delta_NDVI: 0.024
entropy: 0.024
NDVI_1: 0.024
SAVI: 0.023
entropy_1: 0.022
SAVI_1: 0.022
ARVI_1: 0.021


In [834]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(
    rf_model, X, y, cv=5, scoring='f1_macro', n_jobs=-1
)
print("5‑fold F1_macro scores:", scores)
print("Mean:", scores.mean(), "Std:", scores.std())


5‑fold F1_macro scores: [0.92630082 0.90411156 0.97159832 0.94072997 0.94010879]
Mean: 0.9365698932627712 Std: 0.02198098357458017
